## SRJ

1. Use cpq - office pc

## Ground and Excited states - Need to import to qasm

In [1]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import qiskit.qasm2
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    #print('core orbitals:', core)
    #print('active orbitals:', active)
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    
    #Measurements - GS state - Changing to qml.sample()
    dev1 = qml.device("lightning.qubit", wires=qubits) 
    @qml.qnode(dev1)
    def gs_sample(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        #print('Ash excitation  for sample prep', ash_excitation)
        #print('Params for sample prep', params)
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.sample()

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    def callback(params):
        print(f"Current parameters: {params}")
        print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    #print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        #print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value


        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        #print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization



        #Energy calculation
        result = minimize(cost, params, method='powell', callback=callback, tol = 1e-12, options = {'disp': False, 'maxiter': 1e8})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)


        ostate = new_state(hf_state, ash_excitation, params)
        #print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)

    qc = (qml.to_openqasm(gs_sample)(hf_state, ash_excitation, params))
    qcircuit = qiskit.qasm2.loads(qc)  #qc = qasm file
    op_counts = qcircuit.count_ops()
    # Print the counts
    print(op_counts)

    # Access the count of CNOT gates specifically
    cnot_count = op_counts.get('cx', 0)
    print(f"Number of CNOT gates in GS state calcs: {cnot_count}")

    return gs_state, params, ash_excitation, qubits, H, gs_sample, op_counts

import os
from time import time
import pennylane as qml
from pennylane import numpy as np
import scipy
configd = []

#elec: 8, spin-orb: 12
def inite(elec,orb):
    config=[]
    list1=[]
    #singles
    for x in range(elec):
        count=elec
        while (count<orb):
            for e in range(elec):
                if x==e:
                    if x%2==0:
                        config.append(count)
                        count = count+2
                    else:
                        config.append(count+1)
                        count=count+2
                else:
                    config.append(e)
            list1.append(config)
            config=[]
    #print('Len of singles', len(list1))
    #doubles
    for x in range(elec):
        for y in range(x+1,elec):
            count1=orb-elec
            count2=orb-elec
            for count1 in range(elec, orb, 2):
                for count2 in range(elec, orb, 2):
                    cont=0
                    if count1==count2:
                        if (x%2)!=(y%2):
                            cont=1
                    else:
                        cont=1
                    if (x%2)==(y%2) and count2<count1:
                        cont=0
                    if cont==1:    
                        for e in range(elec):
                            if x==e:
                                if x%2==0:
                                    config.append(count1)
                                else:
                                    config.append(count1+1)
                            elif y==e:
                                if y%2==0:
                                    config.append(count2)
                                else:
                                    config.append(count2+1)
                            else:
                                config.append(e)
                        list1.append(config)
                        #print('lenght of total exc:', len(list1))
                        config=[]
    return list1

def ee_exact(symbols, coordinates, active_electrons, active_orbitals ,params,ash_excitation, shots=0):
    print('\n')
    #print('The no of active electrons:', active_electrons)
    #print('The no of active orbitals:', active_orbitals)
    H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf", active_electrons=active_electrons, active_orbitals=active_orbitals)
    hf_state = qml.qchem.hf_state(active_electrons, qubits)
    #print('HF state:', hf_state)
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)
    wires=range(qubits)
    

    null_state = np.zeros(qubits,int)
    #print('Null state is', null_state)
    list1 = inite(active_electrons,qubits)
    #print('The list1 :', list1)
    #print('Len of list1', len(list1))
    values =[]
    for t in range(1):
        if shots==0:
            dev = qml.device("lightning.qubit", wires=qubits)
        else:

            dev = qml.device("lightning.qubit", wires=qubits,shots=shots)
        #circuit for diagonal part
        @qml.qnode(dev)
        def circuit_d(params, occ,wires, hf_state, ash_excitation):
            #print('What is going  as hf_State:', hf_state)
            qml.BasisState(hf_state, wires=range(qubits))
            for w in occ:
                qml.X(wires=w)
            #Going to include excitations here
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        # For Sample measurement
        @qml.qnode(dev)
        def circuit_d_measure(params, occ,wires, hf_state, ash_excitation):
            #print('What is going  as hf_State:', hf_state)
            qml.BasisState(hf_state, wires=range(qubits))
            for w in occ:
                qml.X(wires=w)
            #Going to include excitations here
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.sample()
        
        #circuit for off-diagonal part
        @qml.qnode(dev)
        def circuit_od(params, occ1, occ2,wires, hf_state, ash_excitation):
            qml.BasisState(hf_state, wires=range(qubits))
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        #print('Hadamard applied to',v)
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.expval(H)
        
        @qml.qnode(dev)
        def circuit_od_measure(params, occ1, occ2,wires, hf_state, ash_excitation):
            qml.BasisState(hf_state, wires=range(qubits))
            for w in occ1:
                qml.X(wires=w)
            first=-1
            for v in occ2:
                if v not in occ1:
                    if first==-1:
                        first=v
                        #print('Hadamard applied to',v)
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for v in occ1:
                if v not in occ2:
                    if first==-1:
                        first=v
                        qml.Hadamard(wires=v)
                    else:
                        qml.CNOT(wires=[first,v])
            for i, excitations in enumerate(ash_excitation):
                if len(ash_excitation[i]) == 4:
                    qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
                elif len(ash_excitation[i]) == 2:
                    qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
            return qml.sample()
        #final M matrix
        
        M = np.zeros((len(list1),len(list1)))
        offdiagmat = np.zeros((len(list1),len(list1)))
        diagmat = np.zeros((len(list1),len(list1)))
        op_counts_diag = []
        op_counts_offdiag = []
        had = 0.0
        cnot = 0.0
        had_offdiag = 0.0
        cnot_offdiag = 0.0
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i == j:
                    M[i,i] = circuit_d(params, list1[i], wires, null_state, ash_excitation)
        #print(f'The value of i and j', {i}, {j})
        diagmat = (qml.to_openqasm(circuit_d_measure)(params, list1[i], wires, null_state, ash_excitation))
        qcircuit_diag = qiskit.qasm2.loads(diagmat)  #qc = qasm file

        op_counts_diag = qcircuit_diag.count_ops()
        cnot = op_counts_diag.get('cx', 0)
        had = op_counts_diag.get('h',0)
        print('CNOT in diag matrix', cnot)
        print('Hadamard in diag matrix', had)
        print("diagonal parts done")
        print('\n')
        for i in range(len(list1)):
            for j in range(len(list1)):
                if i!=j:
                    Mtmp = circuit_od(params, list1[i],list1[j],wires, null_state, ash_excitation)
                    M[i,j]=Mtmp-M[i,i]/2.0-M[j,j]/2.0
        #print(f'The value of i and j', {i}, {j})
        offdiagmat = (qml.to_openqasm(circuit_od_measure)(params, list1[i],list1[j],wires, null_state, ash_excitation))
        qcircuit_offdiag = qiskit.qasm2.loads(offdiagmat)  #qc = qasm file

        op_counts_offdiag = qcircuit_offdiag.count_ops()
        cnot_offdiag = op_counts_offdiag.get('cx', 0)
        had_offdiag= op_counts_offdiag.get('h',0)

        #print(op_counts_offdiag)
        print('CNOT in off diag', cnot_offdiag)
        print('Hadamard in off diag', had_offdiag)
        print("off diagonal terms done")
        print('\n')
        #ERROR:not subtracting the gs energy
        eig,evec=np.linalg.eig(M)
        values.append(np.sort(eig))
    return values, cnot, had



In [2]:
symbols  = [ 'H', 'H']
print('H2-0.735A-GS-Powell-sto3g')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 0.735*r_bohr]])



electrons = 2  # 7 from N and 3 from H
orbitals = 4
charge = 0

active_electrons = 2
active_orbitals = 2 #Thinkng it is spatial 



gs_state, params, ash_excitation, qubits, H, gs_sample, op_counts = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1) #1 is used for params





eig, cnot, had = ee_exact(symbols, coordinates,active_electrons,active_orbitals,params,ash_excitation)
print('exact eigenvalues:\n', eig)



H2-0.735A-GS-Powell-sto3g
Using active space, check if you change the H accordingly
HF state is -1.1169989990567457
The adapt iteration now is 0
Current parameters: [0.22353698]
Current cost: -1.137306035765537

Current parameters: [0.22353698]
Current cost: -1.137306035765537

Final updated parameters: [0.22353698]
Final cost: -1.137306035765537
OrderedDict([('cx', 48), ('h', 32), ('rx', 32), ('rz', 8), ('measure', 4), ('x', 2)])
Number of CNOT gates in GS state calcs: 48


CNOT in diag matrix 48
Hadamard in diag matrix 32
diagonal parts done


CNOT in off diag 48
Hadamard in off diag 32
off diagonal terms done


exact eigenvalues:
 [tensor([-0.52461549, -0.1627531 ,  0.49505787], requires_grad=True)]


/Users/s.poyyapakkam/miniconda3/envs/cpq/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


## FCI 

In [3]:
from pyscf import gto, scf, mcscf
import numpy as np
 
# Define the molecule
mol = gto.M(
    atom='H 0.0 0.0 0.0; H 0.0 0.0 0.735;',
    basis='sto-3g',
    verbose=1  # Optional: Adjusts verbosity for output
)
 
# Perform Restricted Hartree-Fock calculation
mf = scf.RHF(mol)
mf.kernel()
 
# Get the HF energy
hf_energy = mf.e_tot
print("Hartree-Fock Ground State Energy:", hf_energy)
 
# Define the active space
ncas = 2  # Number of active orbitals - spatial it seems
nelecas = 2  # Number of electrons in the active space
 
# Set up the CASCI/CASSCF solver
mycas = mcscf.CASCI(mf, ncas, nelecas)
 
 
mycas.fcisolver.nroots=4
 
# Compute the CASCI/CASSCF energy
e_casci = mycas.kernel()
 
# Output the results
print("CASCI/CASSCF Ground State Energy:", e_casci)
print("Hartree-Fock Ground State Energy:", hf_energy)

Hartree-Fock Ground State Energy: -1.1169989967540044
CASCI/CASSCF Ground State Energy: (array([-1.13730604, -0.52461556, -0.16275316,  0.49505774]), array([-1.85727503, -1.24458455, -0.88272215, -0.22491125]), [FCIvector([[ 9.93760401e-01,  2.45740235e-16],
           [ 1.59296347e-16, -1.11535939e-01]]), FCIvector([[ 0.        ,  0.70710678],
           [-0.70710678,  0.        ]]), FCIvector([[ 2.60339513e-16, -7.07106781e-01],
           [-7.07106781e-01, -2.22044605e-16]]), FCIvector([[ 1.11535939e-01, -2.13731789e-16],
           [-9.31241789e-17,  9.93760401e-01]])], array([[ 0.54830202,  1.21832731],
       [ 0.54830202, -1.21832731]]), array([-0.57475062,  0.67074238]))
Hartree-Fock Ground State Energy: -1.1169989967540044
